In [6]:
import os
import typing

# Import your dependencies
import joblib
import pandas as pd
import scipy
import sklearn.metrics
import warnings
from sklearn.exceptions import DataConversionWarning

warnings.filterwarnings("ignore", category=DataConversionWarning)


X_train = pd.read_parquet("X_train.parquet")
y_train = pd.read_parquet("y_train.parquet")

In [8]:
import numpy as np
import pandas as pd
from scipy import stats
from multiprocessing import Pool, cpu_count
import numpy as np, pandas as pd
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline


def extract_features(df):
    # df = DataFrame with multiindex (id, time) and columns ['value', 'period']
    feats = []
    ids = df.index.get_level_values(0).unique()
    for i in ids:
        s = df.loc[i]
        before = s[s['period'] == 0]['value'].values
        after  = s[s['period'] == 1]['value'].values
        def agg(x):
            if len(x)==0:
                return dict(mean=np.nan, std=np.nan, median=np.nan)
            return dict(mean=np.mean(x), std=np.std(x, ddof=1),
                        median=np.median(x),
                        skew=stats.skew(x), kurt=stats.kurtosis(x),
                        q25=np.percentile(x,25), q75=np.percentile(x,75),
                        q10=np.percentile(x,10), q90=np.percentile(x,90),
                        outliers=(np.abs(x - np.mean(x)) > 3*np.std(x)).sum()/len(x))
        b = agg(before); a = agg(after)
        row = {'id': i}
        for k,v in b.items(): row[f'b_{k}'] = v
        for k,v in a.items(): row[f'a_{k}'] = v
        # differences & statistical tests
        row['mean_diff'] = b['mean'] - a['mean']
        row['std_ratio'] = (a['std'] / (b['std'] + 1e-9))
        # t-test and ks (handle degenerate)
        try:
            tstat, tp = stats.ttest_ind(before, after, equal_var=False, nan_policy='omit')
        except:
            tstat, tp = np.nan, np.nan
        row['tstat'] = tstat; row['tp'] = tp
        try:
            ks, ksp = stats.ks_2samp(before, after)
        except:
            ks, ksp = np.nan, np.nan
        row['ks'] = ks
        feats.append(row)
    return pd.DataFrame(feats).set_index('id')

X = extract_features(X_train.loc[:5000])

y = y_train.loc[X.index].astype(int)
# pipeline = Pipeline([('scaler', RobustScaler()), ('clf', RandomForestClassifier(n_estimators=200, class_weight='balanced'))])
# cv = StratifiedKFold(5, shuffle=True, random_state=42)
# print("cv roc_auc:", cross_val_score(pipeline, X.fillna(0), y, cv=cv, scoring='roc_auc').mean())


In [9]:
import numpy as np, pandas as pd
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('scaler', RobustScaler()), ('clf', RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42))])
cv = StratifiedKFold(5, shuffle=True, random_state=42)
print("cv roc_auc:", cross_val_score(pipeline, X.fillna(0), y, cv=cv, scoring='roc_auc').mean())

cv roc_auc: 0.6806121745048486
